In [52]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.11.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")

In [53]:
import com.fasterxml.jackson.annotation.JsonFormat

@JsonFormat(shape = JsonFormat.Shape.ARRAY)
data class Location(
    val latitude: Double,
    val longitude: Double
) {
    fun calcEuclideanDistanceTo(other: Location): Double {
        val xDifference = latitude - other.latitude
        val yDifference = longitude - other.longitude
        return Math.sqrt(xDifference * xDifference + yDifference * yDifference)
    }
}


In [54]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.variable.PlanningId
import com.fasterxml.jackson.annotation.JsonProperty

data class Agent(
    @PlanningId
    val name: String,
    val homeLocation: Location,
    val skills: List<String>
)


Line_54.jupyter.kts (3:52 - 62) Unresolved reference: PlanningId
Line_54.jupyter.kts (7:6 - 16) Unresolved reference: PlanningId

In [55]:
data class Meeting(
    val name: String,
    val location: Location,
    @JsonProperty("required_skill")
    val requiredSkill: String,
    @JsonProperty("time_window")
    val timeWindow: List<Int>, // Assuming time window is represented as [startTime, endTime]
    val duration: Int
)


Line_55.jupyter.kts (4:6 - 18) Unresolved reference: JsonProperty
Line_55.jupyter.kts (6:6 - 18) Unresolved reference: JsonProperty

In [56]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningVariable

@PlanningEntity
data class Assignment(
    val meeting: Meeting
) {
    @PlanningVariable(valueRangeProviderRefs = ["agentRange"], nullable = true)
    var agent: Agent? = null

    // No-arg constructor required for Timefold
    constructor() : this(Meeting("", Location(0.0, 0.0), "", listOf(0, 0), 0))
}


Line_56.jupyter.kts (6:18 - 25) Unresolved reference: Meeting
Line_56.jupyter.kts (9:16 - 21) Unresolved reference: Agent
Line_56.jupyter.kts (12:26 - 33) Unresolved reference: Meeting

In [57]:
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

@PlanningSolution
data class Schedule(
    @ProblemFactCollectionProperty
    @ValueRangeProvider(id = "agentRange")
    val agents: List<Agent>,

    @ProblemFactCollectionProperty
    val meetings: List<Meeting>,

    @PlanningEntityCollectionProperty
    val assignments: List<Assignment>
) {
    @PlanningScore
    var score: HardSoftScore? = null

    // No-arg constructor required for Timefold
    constructor() : this(emptyList(), emptyList(), emptyList())
}


Line_57.jupyter.kts (12:22 - 27) Unresolved reference: Agent
Line_57.jupyter.kts (15:24 - 31) Unresolved reference: Meeting
Line_57.jupyter.kts (18:27 - 37) Unresolved reference: Assignment

In [58]:
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.buildin.hardsoft.HardSoftScore

class AssignmentConstraintProvider : ConstraintProvider {
    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint> {
        return arrayOf(
            requiredSkillConstraint(constraintFactory),
            agentConflictConstraint(constraintFactory),
            minimizeTravelDistanceConstraint(constraintFactory),
            timeWindowConstraint(constraintFactory)
        )
    }

    // Hard constraint: Agent must have the required skill
    private fun requiredSkillConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.from(Assignment::class.java)
            .filter { assignment ->
                val agentSkills = assignment.agent?.skills ?: return@filter false
                !agentSkills.contains(assignment.meeting.requiredSkill)
            }
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Agent must have the required skill")
    }

    // Hard constraint: An agent can be assigned to only one meeting at a time
    private fun agentConflictConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.fromUniquePair(Assignment::class.java)
            .filter { a1, a2 ->
                a1.agent != null && a1.agent == a2.agent &&
                        timeOverlap(a1.meeting, a2.meeting)
            }
            .penalize(HardSoftScore.ONE_HARD)
            .asConstraint("Agent cannot be assigned to overlapping meetings")
    }

    // Soft constraint: Minimize total travel distance
    private fun minimizeTravelDistanceConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory.from(Assignment::class.java)
            .filter { it.agent != null }
            .penalizeDouble(HardSoftScore.ONE_SOFT) { assignment ->
                val agentLocation = assignment.agent!!.homeLocation
                val meetingLocation = assignment.meeting.location
                agentLocation.calcEuclideanDistanceTo(meetingLocation)
            }
            .asConstraint("Minimize travel distance")
    }

    // Hard constraint: Meeting must be within agent's availability (assuming agents are available all the time for now)
    // Alternatively, enforce meeting time windows
    private fun timeWindowConstraint(constraintFactory: ConstraintFactory): Constraint {
        // For simplicity, we assume agents are available at all times.
        // Implement this constraint if agents have specific availability windows.
        return constraintFactory.from(Assignment::class.java)
            .filter { it.agent != null } // Optional: Add time window constraints here
            .reward(HardSoftScore.ZERO)
            .asConstraint("Time window constraint placeholder")
    }

    // Helper function to check time overlap between two meetings
    private fun timeOverlap(meeting1: Meeting, meeting2: Meeting): Boolean {
        val start1 = meeting1.timeWindow[0]
        val end1 = meeting1.timeWindow[1]
        val start2 = meeting2.timeWindow[0]
        val end2 = meeting2.timeWindow[1]
        return start1 < end2 && start2 < end1
    }
}


Line_58.jupyter.kts (18:34 - 38) 'from(Class<A!>!): UniConstraintStream<A!>!' is deprecated. Deprecated in Java
Line_58.jupyter.kts (18:34 - 38) Not enough information to infer type variable A
Line_58.jupyter.kts (18:39 - 49) Unresolved reference: Assignment
Line_58.jupyter.kts (19:23 - 33) Cannot infer a type for this parameter. Please specify it explicitly.
Line_58.jupyter.kts (20:17 - 34) Unreachable code
Line_58.jupyter.kts (20:21 - 32) Variable 'agentSkills' is never used
Line_58.jupyter.kts (21:17 - 72) Unreachable code
Line_58.jupyter.kts (29:34 - 48) 'fromUniquePair(Class<A!>!): BiConstraintStream<A!, A!>!' is deprecated. Deprecated in Java
Line_58.jupyter.kts (29:34 - 48) Not enough information to infer type variable A
Line_58.jupyter.kts (29:49 - 59) Unresolved reference: Assignment
Line_58.jupyter.kts (30:23 - 25) Cannot infer a type for this parameter. Please specify it explicitly.
Line_58.jupyter.kts (30:27 - 29) Cannot infer a type for this parameter. Please specify it ex

In [59]:
import java.io.File
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue

val mapper = jacksonObjectMapper()

// Read the JSON data
data class InputData(
    val vehicles: List<Agent>,
    val visits: List<Meeting>
)

val inputData: InputData = mapper.readValue(File("data.json"))

// Create assignments (initially unassigned)
val assignments = inputData.visits.map { Assignment(it) }

// Create the Schedule (planning solution)
val schedule = Schedule(
    agents = inputData.vehicles,
    meetings = inputData.visits,
    assignments = assignments
)


Line_59.jupyter.kts (9:24 - 29) Unresolved reference: Agent
Line_59.jupyter.kts (10:22 - 29) Unresolved reference: Meeting
Line_59.jupyter.kts (16:36 - 39) Not enough information to infer type variable R
Line_59.jupyter.kts (16:42 - 52) Unresolved reference: Assignment
Line_59.jupyter.kts (19:16 - 24) Unresolved reference: Schedule

In [60]:
import ai.timefold.solver.core.config.solver.SolverConfig
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver

val solverConfig = SolverConfig()
    .withSolutionClass(Schedule::class.java)
    .withEntityClasses(Assignment::class.java)
    .withConstraintProviderClass(AssignmentConstraintProvider::class.java)
    .withTerminationSpentLimit(java.time.Duration.ofSeconds(10))

val solverFactory: SolverFactory<Schedule> = SolverFactory.create(solverConfig)

println("Solving the problem ...")
val solver: Solver<Schedule> = solverFactory.buildSolver()
val solution: Schedule = solver.solve(schedule)
println("Solving finished with score (${solution.score}).")


Line_60.jupyter.kts (6:24 - 32) Unresolved reference: Schedule
Line_60.jupyter.kts (7:24 - 34) Unresolved reference: Assignment
Line_60.jupyter.kts (8:34 - 62) Unresolved reference: AssignmentConstraintProvider
Line_60.jupyter.kts (11:34 - 42) Unresolved reference: Schedule
Line_60.jupyter.kts (14:20 - 28) Unresolved reference: Schedule
Line_60.jupyter.kts (15:15 - 23) Unresolved reference: Schedule
Line_60.jupyter.kts (15:39 - 47) Unresolved reference: schedule

In [61]:
// Collect assignments
val assignmentsByAgent = solution.assignments
    .filter { it.agent != null }
    .groupBy { it.agent!!.name }

println("Assignments:")
assignmentsByAgent.forEach { (agentName, assignments) ->
    println("$agentName is assigned to meetings:")
    assignments.forEach { assignment ->
        println(" - ${assignment.meeting.name} at location ${assignment.meeting.location}")
    }
}


Line_61.jupyter.kts (2:26 - 34) Unresolved reference: solution
Line_61.jupyter.kts (3:15 - 17) Unresolved reference: it
Line_61.jupyter.kts (4:16 - 18) Unresolved reference: it
Line_61.jupyter.kts (7:20 - 27) Overload resolution ambiguity: 
public inline fun <T> Iterable<TypeVariable(T)>.forEach(action: (TypeVariable(T)) -> Unit): Unit defined in kotlin.collections
public inline fun <K, V> Map<out TypeVariable(K), TypeVariable(V)>.forEach(action: (Map.Entry<TypeVariable(K), TypeVariable(V)>) -> Unit): Unit defined in kotlin.collections
Line_61.jupyter.kts (7:30 - 54) Cannot infer a type for this parameter. Please specify it explicitly.
Line_61.jupyter.kts (9:17 - 24) Overload resolution ambiguity: 
public inline fun <T> Iterable<TypeVariable(T)>.forEach(action: (TypeVariable(T)) -> Unit): Unit defined in kotlin.collections
public inline fun <K, V> Map<out TypeVariable(K), TypeVariable(V)>.forEach(action: (Map.Entry<TypeVariable(K), TypeVariable(V)>) -> Unit): Unit defined in kotlin.col